In [1]:
import abc
import pandas as pd
from sqlalchemy.engine import Engine
from utils.utils import easyConnect, upsert_to_mysql
import numpy as np

08-20 15:36:57 | INFO    | utils.logger_config:_setup_logger | 📝 日志系统启动成功
08-20 15:36:57 | INFO    | utils.logger_config:_setup_logger | 📁 主日志文件: C:\Users\31021\Desktop\Quantitative_Trading\Coding\Get_Stock_Infos\logs\20250820-2.log
08-20 15:36:57 | INFO    | utils.logger_config:_setup_logger | 🔴 错误日志文件: C:\Users\31021\Desktop\Quantitative_Trading\Coding\Get_Stock_Infos\logs\20250820-2_error.log
08-20 15:36:57 | INFO    | utils.logger_config:_setup_logger | 📂 项目根目录: C:\Users\31021\Desktop\Quantitative_Trading\Coding\Get_Stock_Infos


In [5]:
df = pd.read_excel('../assets/test.xlsx')

In [6]:
df

,ts_code,trade_date,data1,data2
0,000001.SZ,2024-08-18,2,6
1,000001.SZ,2024-08-19,4,12
2,000001.SZ,2024-08-20,5,15
3,000001.SZ,2024-08-21,6,12
4,000002.SZ,2024-08-18,3,6
5,000002.SZ,2024-08-19,12,12
6,000002.SZ,2024-08-20,6,12
7,000002.SZ,2024-08-21,8,4
8,000002.SZ,2024-08-22,9,3
9,000002.SZ,2024-08-23,10,15


In [12]:
def avg(df, window: int, column: str, how: str):
    """
    在 DataFrame 中添加一列移动平均值
    :param df: DataFrame
    :param window: 移动窗口大小
    :param column: 要计算的列名
    :param how: 平均方式, 'simple' | 'weighted' | 'exponential'
    :return: None (直接修改 df)
    """
    how = how.lower()
    new_col = f"{column}_{how}_ma{window}"

    if how == 'simple':
        df[new_col] = df[column].rolling(window=window, min_periods=1).mean()

    elif how == 'weighted':
        def weighted_moving_average(x):
            weights = np.arange(1, len(x) + 1)  # 权重 1,2,3,...,window
            return np.dot(x, weights) / weights.sum()
        df[new_col] = df[column].rolling(window=window, min_periods=1).apply(weighted_moving_average, raw=True)

    elif how == 'exponential':
        df[new_col] = df[column].ewm(span=window, adjust=False).mean()

    else:
        raise ValueError("how 必须是 'simple'、'weighted' 或 'exponential'")

In [13]:
avg(df, window=2, column='data1', how='simple')
df

,ts_code,trade_date,data1,data2,data1_simple_ma2
0,000001.SZ,2024-08-18,2,6,2.0
1,000001.SZ,2024-08-19,4,12,3.0
2,000001.SZ,2024-08-20,5,15,4.5
3,000001.SZ,2024-08-21,6,12,5.5
4,000002.SZ,2024-08-18,3,6,4.5
5,000002.SZ,2024-08-19,12,12,7.5
6,000002.SZ,2024-08-20,6,12,9.0
7,000002.SZ,2024-08-21,8,4,7.0
8,000002.SZ,2024-08-22,9,3,8.5
9,000002.SZ,2024-08-23,10,15,9.5


In [14]:
pd.read_sql("select ts_code from temp_data", easyConnect())['ts_code'].unique().tolist()

5413

In [15]:
pd.read_sql("select * from temp_data", easyConnect()).head()

,ts_code,trade_date,close,turnover_rate,turnover_rate_f,pe,pe_ttm,pb,ps,ps_ttm,total_share,float_share,free_share,total_mv,circ_mv,udlimit_status
0,002179.SZ,20250818,41.75,1.6302,3.0782,26.3656,27.2681,3.9121,4.2753,4.1113,211827.1050,208070.8850,110195.6796,8.843782e+06,8.686959e+06,0
1,688222.SH,20250818,23.03,5.7899,10.7728,179.6764,140.4483,6.5870,21.6111,21.6439,40068.0000,40068.0000,21534.7310,9.227660e+05,9.227660e+05,0
2,601702.SH,20250818,18.54,1.3589,4.8820,15.2005,15.0438,3.3662,1.7018,1.6210,99853.0600,99853.0600,27793.0000,1.851276e+06,1.851276e+06,0
3,603373.SH,20250818,54.89,10.2533,15.0680,47.0313,46.4874,3.3474,2.2212,2.1836,10752.6882,4543.0108,3091.3979,5.902151e+05,2.493659e+05,0
4,300421.SZ,20250818,16.51,7.1411,8.1025,87.8894,84.6630,3.8654,4.6473,4.5464,29403.0484,23034.4072,20301.2453,4.854443e+05,3.802981e+05,0
